In [1]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

from Artha.nlp_extraction import run_pipeline
import Artha.data_process as dp
from Artha.neodb import Neo
import Artha.crypto_data as crypto
from tqdm import tqdm
import json
import numpy as np
np.set_printoptions(suppress=True)
from datetime import datetime
import pickle
from Artha.pagerank import get_mention_edges
from spacy.tokens import DocBin

# coins database
neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
neo.session.run('''CREATE CONSTRAINT twitter_username IF NOT EXISTS ON (n:Person)
                ASSERT n.username IS UNIQUE''')
neo.session.run('''CREATE CONSTRAINT coin_name IF NOT EXISTS ON (n:Coin)
                ASSERT n.ticker IS UNIQUE''')

CPU times: user 2.42 s, sys: 365 ms, total: 2.79 s
Wall time: 4 s


In [2]:
print(len(neo.get_nodes()), len(neo.get_relations()))
# without mentions 711 6958

509 188


In [3]:
neo.clear_db()

deleted database


In [4]:
neo.load_coins()
# loads everyone's follows
# neo.load_follow_nodes(dp.load_following("checkra_"))

Updated coins


In [5]:
%%time
# add follows between only people I follow, can change by getting rid of second 2 last line for anyone a person follows
follows = []
some_people = ["BTC_JackSparrow", "razoreth", "Nostranomist", "CryptoKaleo", "nebraskangooner", "Rager", "SavageBTC"]
available = [i[1:-4] for i in os.listdir("../data/follows/")]

# generate follow edges for people within people group ^
for user in some_people:
    fs = dp.load_following(user)
    for i in fs:
        if i[3] in available:
            follows.append(i)

CPU times: user 22.2 ms, sys: 4.27 ms, total: 26.5 ms
Wall time: 28.6 ms


In [6]:
%%time
print(len(follows))
neo.load_follow_nodes(follows)
neo.load_follow_relations(follows)

264
loaded People
loaded relations
CPU times: user 5.39 ms, sys: 3.71 ms, total: 9.1 ms
Wall time: 150 ms


In [7]:
%%time
all_tweets = []
for ind, username in tqdm(enumerate(some_people)):
    cur_tweets = dp.load_tweets(username, location = "../data/tweets1/u")
    all_tweets.extend([tweet for tweet in dp.clean_tweets(cur_tweets, username)])

print(len(all_tweets))

7it [00:01,  4.97it/s]60943
CPU times: user 1.34 s, sys: 57.1 ms, total: 1.4 s
Wall time: 1.41 s



In [8]:
%%time
docs = run_pipeline(all_tweets)
print("\n", docs[0]._.tweeted_at)

60943it [00:58, 1047.42it/s]
 05/01/2021 13:08:26
CPU times: user 50.9 s, sys: 3.71 s, total: 54.6 s
Wall time: 58.2 s



In [9]:
all_docs = {}
for doc in tqdm(docs):
    if doc._.username not in all_docs.keys():
        all_docs[doc._.username] = [doc]
        print(doc._.username)
    else:
        all_docs[doc._.username].append(doc)

100%|██████████| 14474/14474 [00:00<00:00, 121509.79it/s]BTC_JackSparrow
razoreth
Nostranomist
CryptoKaleo
nebraskangooner
Rager
SavageBTC



In [10]:
date = "04/03/2021 00:00:00"

all_mentions = []
for username in some_people:
    cur_mentions = get_mention_edges(all_docs[username], username, win_start_date = date)
    all_mentions.extend(cur_mentions)
    print("done", username)

done BTC_JackSparrow
done razoreth
done Nostranomist
done CryptoKaleo
done nebraskangooner
done Rager
done SavageBTC


In [ ]:
%%time
print(len(all_mentions))
neo.load_mentions(all_mentions)

In [ ]:
%%time
neo.load_mentions(all_mentions)

In [ ]:
result = neo.session.run('''
    CALL gds.graph.create(
        'mentions',
        {Person: {label: "Person"},Coin: {label: "Coin"}},
        {CALLOUT: {type : 'CALLOUT', orientation: 'NATURAL'},
            FOLLOWS: {type: 'FOLLOWS', orientation: 'NATURAL'}
        },
        { relationshipProperties: 'weight'}
    )
    YIELD graphName, nodeCount, relationshipCount
    ''')
[r for r in result]

In [ ]:
result = neo.session.run('''
    CALL gds.pageRank.stream('mentions', 
        {
        maxIterations: 20,
        dampingFactor: 0.85,
        relationshipWeightProperty: 'weight'
        })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).ticker AS ticker, score
    ORDER BY score DESC, ticker ASC

''')
f = [r for r in result]
for i in f:
    if i["score"]>.1500001:
        print(i)

In [ ]:
result = neo.session.run("CALL gds.graph.drop('mentions') YIELD graphName")
[r for r in result]

In [ ]:
# make graphview that only looks at people who mention coins, and coins that are mentioned

# exchange tokens might have more mentions
# factor in mentions of coins in threads, so it gets more mentions than just 1